### Assignment3- PartC
User Keras-NLP for various nlp tasks

Inference with a pretrained classifier

Fine tuning a pretrained backbone

Fine tuning with user-controlled preprocessing

Fine tuning a custom modes

### Inference with a pretrained classifier


In [ ]:
!pip install keras-nlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 90.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 26.0 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Suc

In [ ]:
from keras_nlp.models import BertClassifier

# Load a pre-trained classifier
classifier = BertClassifier.from_preset("bert_tiny_en_uncased_sst2")

# Example review
review = "This movie was a fantastic journey through emotions and storytelling!"

# Predict sentiment
# Note: The model expects batched inputs, so we wrap the string in a list
prediction = classifier.predict([review])

# Interpret the prediction
# The output is logits per class in the order [negative, positive]
# You can convert these logits into probabilities with a softmax function if needed

print(prediction)  # This will show you the raw logits

# To interpret the logits, you might simply consider the higher logit to correspond to the predicted class


100%|██████████| 2.14k/2.14k [00:00<00:00, 3.98MB/s]
100%|██████████| 226k/226k [00:00<00:00, 6.31MB/s]
100%|██████████| 16.8M/16.8M [00:00<00:00, 60.5MB/s]


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[-2.5448606  2.6114783]]


In [ ]:
import numpy as np

logits = np.array([-2.5448606, 2.6114783])
probabilities = np.exp(logits) / np.sum(np.exp(logits))

print(probabilities)


[0.00572974 0.99427026]


### Fine tuning a pretrained BERT backbone

In [ ]:
import pandas as pd

# Load the Parquet file
df = pd.read_parquet('/content/train-00000-of-00001.parquet')

# Display the first few rows of the dataframe to understand its structure
print(df.head())


   idx                                           sentence  label
0    0       hide new secretions from the parental units       0
1    1               contains no wit , only labored gags       0
2    2  that loves its characters and communicates som...      1
3    3  remains utterly satisfied to remain the same t...      0
4    4  on the worst revenge-of-the-nerds clichés the ...      0


In [ ]:
df.shape

(67349, 3)

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import text_dataset_from_directory
import keras_nlp

# This assumes you have already loaded your dataframe 'df' from the Parquet file.
sentences = df['sentence'].tolist()
labels = df['label'].tolist()

# Create a dataset from the sentences and labels
dataset = tf.data.Dataset.from_tensor_slices((sentences, labels))

# Batch the dataset
batch_size = 32
dataset = dataset.batch(batch_size)

# Split the dataset into training and validation
# Here, I'm just showing a simple split, but you should use a proper method to split your dataset.
train_size = int(0.8 * len(sentences))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

# Initialize the BertClassifier
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=2,
)

# Compile the model
classifier.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

# Fit the model
classifier.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1,
)


###Fine tuning a pretrained backbone

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import text_dataset_from_directory
import keras_nlp

# This assumes you have already loaded your dataframe 'df' from the Parquet file.
sentences = df['sentence'].tolist()
labels = df['label'].tolist()


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy



In [ ]:
# Create a dataset from the sentences and labels
train_dataset = tf.data.Dataset.from_tensor_slices((sentences, labels))

# Batch the dataset
batch_size = 8  # You can adjust the batch size
train_dataset = train_dataset.batch(batch_size)

# Initialize the BertClassifier
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=2,
    dropout=0.1,
)

classifier.compile(
    optimizer=Adam(learning_rate=3e-5),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],

)

# Fit the model
classifier.fit(
    train_dataset,
    epochs=1,
)

 953/8419 ━━━━━━━━━━━━━━━━━━━━ 1:24:41 681ms/step - accuracy: 0.5998 - loss: 0.6541

KeyboardInterrupt: 

###Fine tuning with user-controlled preprocessing


In [ ]:
import tensorflow as tf
import keras_nlp
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Let's assume df is your loaded DataFrame from the Parquet file

# Convert DataFrame columns to a list of dictionaries (one per example)
# and split the data into a training set and a validation set
train_sentences = df['sentence'].tolist()
train_labels = df['label'].tolist()

# Create TensorFlow datasets from the sentences and labels
train_data = tf.data.Dataset.from_tensor_slices((train_sentences, train_labels))

# Set your batch size
batch_size = 32

# Shuffle and batch the datasets
train_data = train_data.shuffle(len(train_sentences)).batch(batch_size)

# Initialize the BertPreprocessor
preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_tiny_en_uncased",
    sequence_length=128  # Adjust sequence_length to your needs
)

# Apply the preprocessor to the dataset
train_data = train_data.map(lambda x, y: (preprocessor(x), y),
                            num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Cache and prefetch for performance optimization
train_data = train_data.cache().prefetch(tf.data.experimental.AUTOTUNE)

# Split your dataset into training and validation
# Here we'll use 80% for training and 20% for validation
train_size = int(0.8 * len(train_sentences))
val_size = len(train_sentences) - train_size

train_dataset = train_data.take(train_size)
val_dataset = train_data.skip(train_size)

# Initialize the BertClassifier without passing a preprocessor, as we have done it already
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=2,
    preprocessor=None,  # Preprocessing has been handled manually
)

# Compile the model
classifier.compile(
    optimizer=Adam(learning_rate=3e-5),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Fit the model on the train dataset and validate on the validation dataset
classifier.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
)


Epoch 1/3
 904/2105 ━━━━━━━━━━━━━━━━━━━━ 9:29 474ms/step - accuracy: 0.6586 - loss: 0.6041

KeyboardInterrupt: 

###Custom

In [ ]:
import tensorflow as tf
import keras_nlp
import keras
from keras_nlp.models import BertBackbone
from keras_nlp.layers import TransformerEncoder
from keras.models import Model
from keras.optimizers import AdamW
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy

# Let's assume df is your loaded DataFrame from the Parquet file

# Convert DataFrame columns to a list of dictionaries (one per example)
train_sentences = df['sentence'].tolist()
train_labels = df['label'].tolist()

# Create TensorFlow datasets from the sentences and labels
train_dataset = tf.data.Dataset.from_tensor_slices((train_sentences, train_labels))

# Set your batch size
batch_size = 32

# Shuffle and batch the datasets
train_dataset = train_dataset.shuffle(len(train_sentences)).batch(batch_size)

# Initialize the BertPreprocessor
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_tiny_en_uncased")

# Apply the preprocessor to the dataset
train_dataset = train_dataset.map(lambda x, y: (preprocessor(x), y),
                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Cache and prefetch for performance optimization
train_dataset = train_dataset.cache().prefetch(tf.data.experimental.AUTOTUNE)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(train_sentences))
train_dataset = train_dataset.take(train_size)
val_dataset = train_dataset.skip(train_size)

# Initialize the BertBackbone
backbone = BertBackbone.from_preset("bert_tiny_en_uncased")

# Make the backbone non-trainable
backbone.trainable = False

# Define custom architecture
inputs = backbone.input
sequence = backbone(inputs)["sequence_output"]
for _ in range(2):
    sequence = TransformerEncoder(
        num_heads=2,
        intermediate_dim=512,
        dropout=0.1,
    )(sequence)

# Use [CLS] token output to classify
outputs = keras.layers.Dense(2)(sequence[:, backbone.cls_token_index, :])

# Create the model
model = Model(inputs, outputs)

# Compile the model
model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=AdamW(learning_rate=5e-5),
    metrics=[SparseCategoricalAccuracy()],
    jit_compile=True,
)

# Fit the model on the train dataset and validate on the validation dataset
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3,
)

# Show the model summary
model.summary()


Epoch 1/3
 126/2105 ━━━━━━━━━━━━━━━━━━━━ 1:36:45 3s/step - loss: 0.7739 - sparse_categorical_accuracy: 0.5560

KeyboardInterrupt: 